In [1]:
import probRobScene
import numpy as np
from pyrep import PyRep
from pyrep.objects.object import Object
from probRobScene.wrappers.coppelia.prbCoppeliaWrapper import cop_from_prs

In [2]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

# Places camera at a position and points it towards the origin
def lookFrom(cameraPos, target = [0, 0, 0], up = [0, 0, 1]):
    defaultCamera = Object.get_object('DefaultCamera')
    defaultCamera.set_position(cameraPos)
    
    cameraPos = np.array(cameraPos)
    target = np.array(target)
    up = np.array(up)
    
    zaxis = normalize(target - cameraPos)    
    xaxis = normalize(np.cross(up, zaxis))
    yaxis = np.cross(zaxis, xaxis)
    
    # Arrange transformation matrix according to Coppelia's documentation
    # https://www.coppeliarobotics.com/helpFiles/en/positionOrientationTransformation.htm
    camToWorld = np.column_stack((xaxis, yaxis, zaxis, cameraPos))
    camToWorld = np.row_stack((camToWorld, np.array([0,0,0,1])))
    
    defaultCamera.set_matrix(camToWorld)

In [3]:
def takePic(filepath, position=None):
    scenario = probRobScene.scenario_from_file(filepath)
    
    # Launch an empty scene in PyRep
    pr = PyRep()
    pr.launch("emptyVortex.ttt")

    # ProbRobScene to generate objects
    ex_world, used_its = scenario.generate()
    c_objs = cop_from_prs(pr, ex_world)
    
    # Import LUA code/model to save feed from camera 
    screenshotSensor = pr.import_model("screenshot.ttm")
#     defaultCamera = Object.get_object('DefaultCamera')
    
    if position:
        lookFrom(position)
    pr.script_call("handleStuff@screenshotSensor", 6) # Saves it as screenshot-date-time  
    pr.shutdown()

In [ ]:
#All of this uses CopeliaSim version 4.1

In [4]:
takePic("scenarios/gearInsert.prs", position = [-2, 2, 4])